# The paleopiezometry module

TODO

> **Never combine means -or any other statistic- as if they were datapoints**. It is very tempting to take different separate palaeopiezometric estimates and then take the mean of the means (i.e. the piezometric estimate itself is derived from an average) as if they were data points. You may be surprised to read this and think that you then have to consider the raw data from all the experiments and make the estimate from there. The bad news is that neither approach is correct. The first approach ignores the uncertainty in each estimate of the mean and the second ignores the uncertainty within and between experimental conditions (or measured areas). So what is the correct procedure? It is to take the weighted mean of the means, where each mean is weighted by its variance. This is known as the Mantel-Haenszel method.